In [8]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
from matplotlib.pyplot import figure, show
import matplotlib.patches as patches
import copy
import numpy as np
import common
import os
import json
import pandas as pd

In [2]:
# crafter.

In [3]:
def print_scores(inpaths, legend, budget=1e6, sort=False):
  runs = common.load_runs(inpaths, budget)
  percents, methods, seeds, tasks = common.compute_success_rates(runs, budget)
  scores = common.compute_scores(percents)
  if not legend:
    methods = sorted(set(run['method'] for run in runs))
    legend = {x: x.replace('_', ' ').title() for x in methods}

  scores = scores[np.array([methods.index(m) for m in legend.keys()])]
  means = np.nanmean(scores, -1)
  stds = np.nanstd(scores, -1)

  print('')
  print(r'\textbf{Method} & \textbf{Score} \\')
  print('')
  for method, mean, std in zip(legend.values(), means, stds):
    mean = f'{mean:.1f}'
    mean = (r'\o' if len(mean) < 4 else ' ') + mean
    print(rf'{method:<25} & ${mean} \pm {std:4.1f}\%$ \\')
  print('')

In [4]:
base = '../logs/22_09_09-18_20_all_seed'
input_path = os.listdir(base)
full_input_path = [os.path.join(base, p, 'crafter-episodes') for p in input_path if 'ucb_c_0.3' in p and 'batch-size_64' in p]

In [5]:
# base
full_episode_path = []
for p in full_input_path:
    for j in os.listdir(p):
        if 'jsonl-7' in j:
            full_episode_path.append(os.path.join(p, j))
print(full_episode_path)

['../logs/22_09_09-18_20_all_seed/22_09_09-18_20_all_seed_batch-size_64_seed_6_ucb_c_0.3/crafter-episodes/stats.jsonl-7-20220910T013456', '../logs/22_09_09-18_20_all_seed/22_09_09-18_20_all_seed_batch-size_64_seed_7_ucb_c_0.3/crafter-episodes/stats.jsonl-7-20220910T013455', '../logs/22_09_09-18_20_all_seed/22_09_09-18_20_all_seed_batch-size_64_seed_3_ucb_c_0.3/crafter-episodes/stats.jsonl-7-20220910T013457', '../logs/22_09_09-18_20_all_seed/22_09_09-18_20_all_seed_batch-size_64_seed_4_ucb_c_0.3/crafter-episodes/stats.jsonl-7-20220910T013500', '../logs/22_09_09-18_20_all_seed/22_09_09-18_20_all_seed_batch-size_64_seed_0_ucb_c_0.3/crafter-episodes/stats.jsonl-7-20220910T013502', '../logs/22_09_09-18_20_all_seed/22_09_09-18_20_all_seed_batch-size_64_seed_1_ucb_c_0.3/crafter-episodes/stats.jsonl-7-20220910T013503', '../logs/22_09_09-18_20_all_seed/22_09_09-18_20_all_seed_batch-size_64_seed_2_ucb_c_0.3/crafter-episodes/stats.jsonl-7-20220910T013455', '../logs/22_09_09-18_20_all_seed/22_09_0

In [9]:
# with open(full_episode_path[0], 'r') as json_file:
#     json_list = list(json_file)

df = pd.read_json(path_or_buf=full_episode_path[0], lines=True)

In [10]:
df

,length,reward,achievement_collect_coal,achievement_collect_diamond,achievement_collect_drink,achievement_collect_iron,achievement_collect_sapling,achievement_collect_stone,achievement_collect_wood,achievement_defeat_skeleton,...,achievement_make_iron_sword,achievement_make_stone_pickaxe,achievement_make_stone_sword,achievement_make_wood_pickaxe,achievement_make_wood_sword,achievement_place_furnace,achievement_place_plant,achievement_place_stone,achievement_place_table,achievement_wake_up
0,158,7.1,0,0,2,0,2,0,2,0,...,0,0,0,0,0,0,2,0,1,1
1,208,8.1,0,0,0,0,4,2,7,0,...,0,0,0,1,0,0,2,1,2,1
2,227,8.1,0,0,0,0,1,12,6,0,...,0,0,0,1,0,0,1,11,1,1
3,405,9.1,0,0,44,0,5,1,6,0,...,0,0,0,1,0,0,1,1,1,1
4,162,10.1,2,0,0,0,1,2,7,0,...,0,0,0,1,2,0,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,369,11.1,1,0,1,0,1,23,6,0,...,0,0,0,1,1,0,1,18,1,1
546,223,8.1,0,0,3,0,1,0,10,0,...,0,0,0,1,0,0,1,0,2,2
547,363,11.1,0,0,2,0,1,14,4,0,...,0,0,0,1,1,0,1,14,1,1
548,277,6.1,0,0,13,0,3,0,4,0,...,0,0,0,0,0,0,3,0,2,1


In [13]:
df.columns

Index(['length', 'reward', 'achievement_collect_coal',
       'achievement_collect_diamond', 'achievement_collect_drink',
       'achievement_collect_iron', 'achievement_collect_sapling',
       'achievement_collect_stone', 'achievement_collect_wood',
       'achievement_defeat_skeleton', 'achievement_defeat_zombie',
       'achievement_eat_cow', 'achievement_eat_plant',
       'achievement_make_iron_pickaxe', 'achievement_make_iron_sword',
       'achievement_make_stone_pickaxe', 'achievement_make_stone_sword',
       'achievement_make_wood_pickaxe', 'achievement_make_wood_sword',
       'achievement_place_furnace', 'achievement_place_plant',
       'achievement_place_stone', 'achievement_place_table',
       'achievement_wake_up'],
      dtype='object')

In [14]:
achievement_names = [a for a in df.columns if 'achievement' in a]

In [15]:
len(achievement_names)

22

In [25]:
achievement_df = df[achievement_names]

In [26]:
achievement_np = achievement_df.to_numpy()

In [27]:
achievement_np = np.clip(achievement_np, 0, 1)

In [28]:
achievement_np = achievement_np.mean(axis=0) * 100
achievement_np

array([13.27272727,  0.        , 61.81818182,  0.        , 97.09090909,
       46.90909091, 97.45454545,  4.54545455, 72.18181818, 69.09090909,
        0.18181818,  0.        ,  0.        ,  0.54545455,  1.63636364,
       76.36363636, 25.09090909,  0.36363636, 96.18181818, 44.90909091,
       94.        , 95.45454545])

In [ ]:
def log_transform(x):
    return np.exp(np.log(1 + x).mean()) - 1

In [30]:
score = np.exp(np.log(1 + achievement_np).mean()) - 1
score

12.413243402258406

In [31]:
def compute_score_from_path(path):
    df = pd.read_json(path_or_buf=path, lines=True)
    achievement_names = [a for a in df.columns if 'achievement' in a]
    achievement_df = df[achievement_names]
    achievement_np = achievement_df.to_numpy()
    achievement_np = np.clip(achievement_np, 0, 1)
    achievement_np = achievement_np.mean(axis=0) * 100
    return np.exp(np.log(1 + achievement_np).mean()) - 1

In [32]:
all_scores = [compute_score_from_path(p) for p in full_episode_path]

In [35]:
np.mean(all_scores)

10.005392853113134

In [36]:
np.std(all_scores)

1.9137340842662824

In [37]:
def compute_return_from_path(path):
    df = pd.read_json(path_or_buf=path, lines=True)
    achievement_df = df['reward']
    return np.mean(achievement_df)

In [38]:
all_scores = [compute_return_from_path(p) for p in full_episode_path]

In [39]:
print(np.mean(all_scores))
print(np.std(all_scores))

7.733497161040056
0.5529828232426124
